In [1]:
import os
from tensorflow.keras.models import load_model

model_dir = 'data/saved_models'
# Load the models
generator_loaded = load_model(os.path.join(model_dir, 'generator_model.h5'))
discriminator_loaded = load_model(os.path.join(model_dir, 'discriminator_model.h5'))
gan_loaded = load_model(os.path.join(model_dir, 'gan_model.h5'))

print("Models loaded successfully!")

Models loaded successfully!


In [2]:
from tensorflow.keras.optimizers import Adam

# Recompile discriminator if needed for further training
discriminator_loaded.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002), metrics=['accuracy'])

# Freeze the discriminator within the GAN model and recompile the GAN
discriminator_loaded.trainable = False
gan_loaded.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002))

print("Models recompiled and ready for training!")

Models recompiled and ready for training!


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers

data = pd.read_csv(r'data\master\telecom_customer_data.csv')
data.head()

,CustomerID,Tenure,Contract,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,TechSupport,StreamingTV,StreamingMovies,PaymentMethod,SeniorCitizen,PaperlessBilling,Churn
0,6,50,One year,35.65,4091.54,DSL,Yes,No,Yes,No,Mailed check,False,True,False
1,10,32,Two year,30.45,5257.94,Fiber optic,Yes,No,Yes,Yes,Bank transfer,False,False,True
2,3,11,Month-to-month,97.55,1749.50,Fiber optic,No,Yes,No,Yes,Bank transfer,True,False,False
3,10,17,Month-to-month,49.43,1287.60,DSL,No,No,No,Yes,Mailed check,True,True,True
4,10,70,Two year,35.60,7725.76,NaN,No,No,No,Yes,Mailed check,False,False,False


In [5]:
contract_mapping = {'One year' : 0, 'Two year' : 1, 'Month-to-month' : 2}
internet_service_mapping = {'DSL' : 0, 'Fiber optic' : 1}
payment_mapping = {'Mailed check' : 0, 'Bank transfer' : 1, 'Credit card' : 2, 'Electronic check' : 3}
agree_mapping = {'Yes': 0, 'No' :1}

def labelling(data):
    global contract_mapping, internet_service_mapping, payment_mapping, agree_mapping

    data['Contract'] = data['Contract'].map(contract_mapping)
    data['InternetService'] = data['InternetService'].map(internet_service_mapping)
    data['PaymentMethod'] = data['PaymentMethod'].map(payment_mapping)
    data['OnlineSecurity'] = data['OnlineSecurity'].map(agree_mapping)
    data['TechSupport'] = data['TechSupport'].map(agree_mapping)
    data['StreamingTV'] = data['StreamingTV'].map(agree_mapping)
    data['StreamingMovies'] = data['StreamingMovies'].map(agree_mapping)

    return data

data = labelling(data)

In [6]:
data['InternetService'] = data['InternetService'].fillna(data['InternetService'].mean())
data.fillna(data.mean(), inplace=True)
# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

In [7]:
def delabel(df):
    # Reverse mappings
    contract_reverse_mapping = {v: k for k, v in contract_mapping.items()}
    internet_service_reverse_mapping = {v: k for k, v in internet_service_mapping.items()}
    payment_reverse_mapping = {v: k for k, v in payment_mapping.items()}
    agree_reverse_mapping = {v: k for k, v in agree_mapping.items()}
    
    # Apply the reverse mappings column by column, after rounding values
    df['Contract'] = df['Contract'].round().astype('int').map(contract_reverse_mapping)
    df['InternetService'] = df['InternetService'].round().astype('int').map(internet_service_reverse_mapping)
    df['PaymentMethod'] = df['PaymentMethod'].round().astype('int').map(payment_reverse_mapping)
    df['OnlineSecurity'] = df['OnlineSecurity'].round().astype('int').map(agree_reverse_mapping)
    df['TechSupport'] = df['TechSupport'].round().astype('int').map(agree_reverse_mapping)
    df['StreamingTV'] = df['StreamingTV'].round().astype('int').map(agree_reverse_mapping)
    df['StreamingMovies'] = df['StreamingMovies'].round().astype('int').map(agree_reverse_mapping)
    df['SeniorCitizen'] = df['SeniorCitizen'].round().astype('int')
    df['PaperlessBilling'] = df['PaperlessBilling'].round().astype('int')
    df['Churn'] = df['Churn'].round().astype('int')
    df['CustomerID'] = df['CustomerID'].round().astype('int')
    df['Tenure'] = df['Tenure'].round().astype('int')
    
    return df

In [8]:
# Generate synthetic data using the loaded generator model
def generate_synthetic_data(generator, num_samples=1000):
    noise = np.random.normal(0, 1, size=(num_samples, latent_dim))
    synthetic_data = generator.predict(noise)
    return scaler.inverse_transform(synthetic_data)  # Rescale back to original data range

# Generate a new synthetic dataset
latent_dim = 10
synthetic_data = generate_synthetic_data(generator_loaded)
synthetic_df = pd.DataFrame(synthetic_data, columns=data.columns)

# Convert labels back to original categories if needed
synthetic_df = delabel(synthetic_df)  # Assuming delabel function is defined
synthetic_df.head()


32/32 [==============================] - 0s 2ms/step


,CustomerID,Tenure,Contract,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,TechSupport,StreamingTV,StreamingMovies,PaymentMethod,SeniorCitizen,PaperlessBilling,Churn
0,10,71,Month-to-month,62.362514,8990.160156,Fiber optic,No,Yes,No,Yes,Electronic check,0,1,0
1,2,9,One year,45.273777,8773.561523,DSL,No,Yes,No,Yes,Mailed check,0,0,1
2,3,9,Month-to-month,48.397861,8988.458008,Fiber optic,Yes,Yes,No,No,Bank transfer,0,0,1
3,9,61,Month-to-month,59.992470,8989.958008,Fiber optic,Yes,Yes,No,Yes,Electronic check,1,1,1
4,2,4,Month-to-month,31.359558,8990.158203,Fiber optic,Yes,Yes,No,No,Electronic check,0,0,1
